In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
device = "cuda"

In [2]:
from __future__ import print_function

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
from torchsummary import summary
import time

c:\Users\admin\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
def evaluate(model, X, Y, params = ["fmeasure"]):
    torch.cuda.empty_cache()
    results = [] 
    predicted = []
    batch_size=1

    
    for i in range(int((len(X)/batch_size))):
        s = i*batch_size
        e = i*batch_size+batch_size
            
        inputs = Variable(X[s:e])
        pred = model(inputs)
            
        predicted.append(pred.data.cpu().numpy())
            
    predicted = torch.Tensor(predicted).view(-1, 1)

    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results



In [4]:

X_train = torch.randn([14231, 9000, 18])
Y_train = torch.ones([14231, 1])
X_val = torch.randn([1581, 9000, 18])
Y_val = torch.ones([1581, 1])
X_test = torch.randn([10249, 9000, 18])
Y_test = torch.ones([10249, 1])


In [5]:
np.shape(X_train)

torch.Size([14231, 9000, 18])

In [6]:
class LSTM(nn.Module):  
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # self.bn = nn.BatchNorm2d(hidden_size)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        """
        shape of x: (N, T, C)  N: batch_size, T: seq_len(time), C: input_size(channel)
        """
        lstm_out, (h, c) = self.lstm(x, None)
        # x = self.bn(lstm_out)
        logits = self.fc(lstm_out[:, -1, :])
        probas = F.softmax(logits, dim=1)
        return probas

if __name__ == "__main__":
    x = torch.randn((1, 9000, 18)).cuda()
    model = LSTM(1, 18, 20, 20).cuda()
    #summary(model.cuda(), (4, 1, 64, 256))
    print(model(x))

tensor([[1.]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [7]:
batch_size = 90
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)
torch.cuda.empty_cache()



for epoch in range(3):  # loop over the dataset multiple times
    print("\nEpoch ", epoch)
    running_loss = 0.0
    for i in range(int(len(X_train)/batch_size-1)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(X_train[s:e].cuda())
        labels = Variable(Y_train[s:e].cuda())



        optimizer.zero_grad()
        outputs = model.forward(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        #print(loss.data)


Epoch  0

Epoch  1

Epoch  2


In [10]:
model.forward(X_train.cuda()[5:15])

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:0', grad_fn=<SoftmaxBackward0>)